In [ ]:
imageDirectory = "/content/drive/pathToWhereImagesAre/"
inferencesDirectory = "/content/drive/pathToWhereYouWantInferencesToGo/"

In [ ]:
import cv2
import os
import numpy as np
import pandas as pd
from scipy import signal
import matplotlib.pyplot as plt
from google.colab import drive
from google.colab.patches import cv2_imshow
drive.mount('/content/drive', force_remount = True)

def getBorders(cv2Image):
  src = cv2.cvtColor(cv2Image, cv2.COLOR_BGR2GRAY)
  src = cv2.blur(src,(src.shape[1], 1))
  src = cv2.adaptiveThreshold(src,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,101,2)
  # cv2_imshow(src)
  points = np.sum(src, axis = 1)
  points = signal.savgol_filter(points, 300, 2)
  # plt.plot(range(len(points)), points)
  peaks, _ = signal.find_peaks(points, distance = 1000)
  # plt.plot(peaks[0], points[peaks[0]], "x")
  # plt.plot(peaks[-1], points[peaks[-1]], "x")
  row1 = peaks[0]
  row2 = peaks[-1]
  src = cv2.cvtColor(cv2Image, cv2.COLOR_BGR2GRAY)
  src = cv2.blur(src,(1, src.shape[0]))
  src = cv2.adaptiveThreshold(src,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,101,2)
  # cv2_imshow(src)
  points = np.sum(src, axis = 0)
  points = signal.savgol_filter(points, 300, 2)
  # plt.plot(range(len(points)), points)
  peaks, _ = signal.find_peaks(points, distance = 700)
  # plt.plot(peaks[0], points[peaks[0]], "x")
  # plt.plot(peaks[-1], points[peaks[-1]], "x")
  column1 = peaks[0]
  column2 = peaks[-1]
  if(abs((column2 - column1) - 4365) > 200):
    print("col corrected")
    column1 = 227
    column2 = 4643
  if(abs((row2 - row1) - 2100) > 150):
    print("row corrected")
    row1 = 725
    row2 = 2979
  return([row1, row2, column1, column2])

Mounted at /content/drive


In [ ]:
if("inferencesInProgress.csv" in os.listdir(inferencesDirectory)):
  inferences = pd.read_csv(inferencesDirectory + "inferencesInProgress.csv")
else:
  inferences = pd.DataFrame(columns = ['imageCode', 'count'])
print(inferences)

Empty DataFrame
Columns: [imageCode, count]
Index: []


In [ ]:
for file in os.listdir(imageDirectory):

    thisImageCode = str(os.path.splitext(os.path.basename(file))[0])
    if((thisImageCode + ".jpg") in list(inferences['imageCode'])):
      continue

    print(file)

    full_file = os.path.join(imageDirectory, file)

    img = cv2.imread(full_file)
    thisBorders = getBorders(img)
    cv2.rectangle(img, (thisBorders[2], thisBorders[0]), (thisBorders[3], thisBorders[1]), 1, 10)

# Convert to grayscale for binarization later
    img_gray = cv2.imread(full_file, cv2.IMREAD_GRAYSCALE)

# Binarize image with adaptive threshold
    thres = cv2.adaptiveThreshold(img_gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                cv2.THRESH_BINARY_INV,101,2)

# TUNE: KERNEL SIZE
# Preprocess image, morphologically open image, removes small spots
    kernel = np.ones((7, 7), np.uint8)
    thres = cv2.morphologyEx(thres, cv2.MORPH_OPEN, kernel)

# Get all contours
    cnts, _ = cv2.findContours(thres, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    cnts_filtered = []

    for c in cnts:
        area = cv2.contourArea(c)

        # TUNE: AREA THRESHOLD
        # Filter by area
        if(area < 3500 and area > 1700):
            width = min(cv2.minAreaRect(c)[1])
            length = max(cv2.minAreaRect(c)[1])

            # TUNE: L, W, LW THRESHOLD
            if(length > 60 and length < 110 and length/width > 1.5 and length/width < 4):
                solidity = float(area)/cv2.contourArea(cv2.convexHull(c))

                # TUNE: SOLIDITY
                # Solidity filters out bumpy, non-round parts
                if(solidity > 0.8):

                    smoothness = cv2.arcLength(cv2.convexHull(c), True)/ cv2.arcLength(c, True)

                    if(smoothness > 0.9):

                      canvas = np.empty_like(thres)
                      cv2.drawContours(canvas, [c], 0, color=255, thickness=-1)
                      pts = np.where(canvas == 255)
                      egg_opacity = np.median(img_gray[pts[0], pts[1]])

                      # TUNE: EGG COLOR
                      # Egg opacity is how white or black the ROI is
                      if(egg_opacity < 220):
                        objectIsInFrame = (
                          thisBorders[0] < c.flatten()[1] < thisBorders[1] and
                          thisBorders[2] < c.flatten()[0] < thisBorders[3]
                          )
                        if(objectIsInFrame):
                          cnts_filtered.append(c)
                          # print(width, length, area, solidity, egg_opacity)

    inferences.loc[inferences.shape[0]] = [file, len(cnts_filtered)]

    thres = cv2.resize(thres, None, fx = 0.1, fy = 0.1)
    # canvas = np.empty_like(thres)
    # cv2.drawContours(canvas, cnts_filtered, -1, (255,0, 0), 3)
    # canvas = cv2.resize(canvas, None, fx = 0.1, fy = 0.1)
    cv2.drawContours(img, cnts_filtered, -1, (0,255, 20), 6)
    img = cv2.putText(img, 'Egg count: ' + str(len(cnts_filtered)), (100, 100), 1, 8, 1, 10)
    # img = cv2.resize(img, None, fx = 0.1, fy = 0.1)

    cv2.imwrite(inferencesDirectory + file, img)
    inferences.to_csv(inferencesDirectory + 'inferencesInProgress.csv', index = False)

print("done")
inferences.to_csv(inferencesDirectory + 'inferences.csv', index = False)

0.5_Javanica_Unsucrosed_0013.jpg
0.5_Javanica_Unsucrosed_0012.jpg
0.5_Javanica_Unsucrosed_0011.jpg
0.5_Javanica_Unsucrosed_0010.jpg
0.5_Javanica_Unsucrosed_0014.jpg
0.5_Javanica_Unsucrosed_0015.jpg
row corrected
0.5_Javanica_Unsucrosed_0016.jpg
0.25_Javanica_Unsucrosed_0001.jpg
0.25_Javanica_Unsucrosed_0002.jpg
0.25_Javanica_Unsucrosed_0003.jpg
0.25_Javanica_Unsucrosed_0004.jpg
0.25_Javanica_Unsucrosed_0005.jpg
0.25_Javanica_Unsucrosed_0006.jpg
row corrected
0.25_Javanica_Unsucrosed_0007.jpg
0.25_Javanica_Unsucrosed_0008.jpg
debris_Javanica_Unsucrosed_0019.jpg
debris_Javanica_Unsucrosed_0020.jpg
row corrected
debris_Javanica_Unsucrosed_0021.jpg
debris_Javanica_Unsucrosed_0022.jpg
row corrected
debris_Javanica_Unsucrosed_0023.jpg
debris_Javanica_Unsucrosed_0024.jpg
row corrected
debri_Javanica_Unsucrosed_0025.jpg
debris_Javanica_Unsucrosed_0026.jpg
undiluted_Javanica_Unsucrosed_0001.jpg
undiluted_Javanica_Unsucrosed_0002.jpg
undiluted_Javanica_Unsucrosed_0003.jpg
undiluted_Javanica_Unsu